# T2D benchmark

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
os.chdir('../../')

In [2]:
import takco
conf = takco.Config('resources/config-dbpedia.toml')
t2dv2 = takco.TableSet.dataset('t2d-v2', config=conf)

takco.preview(t2dv2)

?,0,1,2
,Title,Author,Source
,Adventures of Huckleberry Finn,Mark Twain,ALA [11]
,The Adventures of Super Diaper Baby,Dav Pilkey,ALA [47]
,The Adventures of Tom Sawyer,Mark Twain,ALA
,Alice series,Phyllis Reynolds Naylor,ALA [2]
,All the King's Men,Robert Penn Warren,Rad
?,0,1,2
,#,Media,MIX
,1,Dainik Jagran,27.500
,2,Dainik Bhaskar,14.000


In [11]:
import glob

searcher_config = {
    "class": "SQLiteSearcher",
    "files" : "data/dbpedia-t2ksubset-label-sqlitedbs-2/*",
    "baseuri": "http://dbpedia.org/resource/",
}

pred = takco.TableSet.link(
    takco.HashBag(list(t2dv2)[:5]),
    usecols = "non_numeric_cols",
    lookup_config = None,
    linker_config = takco.Config({
        "class": "First",
        "searcher": searcher_config,
    }),
)
takco.preview(pred)

?,0,1,2
,Title,Author,Source
,Adventures of Huckleberry Finn,Mark Twain,ALA [11]
,The Adventures of Super Diaper Baby,Dav Pilkey,ALA [47]
,The Adventures of Tom Sawyer,Mark Twain,ALA
,Alice series,Phyllis Reynolds Naylor,ALA [2]
,All the King's Men,Robert Penn Warren,Rad
?,0,1,2
,#,Media,MIX
,1,Dainik Jagran,27.500
,2,Dainik Bhaskar,14.000


In [20]:
from takco.link import *
search = takco.Config(searcher_config).init_class(**globals())
search.search_entities('Central Plaza')

[SearchResult('http://dbpedia.org/resource/Central_Plaza_(Hong_Kong)', {}, score=1.0)]

In [10]:
!time takco run -v -C resources/config-dbpedia.toml resources/pipelines/t2d-v2-baseline.toml

INFO:root:Set log level to 20
INFO:root:Loaded config {'name': 'config-dbpedia', 'datadir': 'data', 'resourcedir': 'resources', 'cache': False, 'kbs': [{'name': 'dbpedia_t2ksubset', 'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia_t2ksubset/db/'}}, {'name': 'dbpedia_2014_part2', 'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia/2014_part2/'}}, {'name': 'dbpedia_2016', 'class': 'RDFSearcher', 'labelProperties': ['http://dbpedia.org/ontology/title'], 'store': {'class': 'Trident', 'configuration': 'kb/dbpedia/2016-10/'}}, {'name': 'dbpedialookup', 'class': 'DBpediaLookup'}], 'assets': [{'name': 't2d-v1', 'class': 'T2D', 'download': ['http://webdatacommons.org/webtables/tables_instance.tar.gz', 'http://webdatacommons.org/webtables/entities_instance.tar.gz', 'http://webdatacommons.org/webtables/classes_instance